In [1]:
import os
import mlflow
import pandas as pd
import psycopg
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, log_loss
from catboost import CatBoostRegressor, CatBoostClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import CatBoostEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [2]:
os.environ["DB_DESTINATION_HOST"] = os.getenv("DB_DESTINATION_HOST")
os.environ["DB_DESTINATION_PORT"] = os.getenv("DB_DESTINATION_PORT")
os.environ["DB_DESTINATION_NAME"] = os.getenv("DB_DESTINATION_NAME")
os.environ["DB_DESTINATION_USER"] = os.getenv("DB_DESTINATION_USER")
os.environ["DB_DESTINATION_PASSWORD"] = os.getenv("DB_DESTINATION_PASSWORD")

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = "real_churn_Andrey"
RUN_NAME = "real_churn_run"
REGISTRY_MODEL_NAME = "real_churn_model_Andrey"


In [3]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.environ["DB_DESTINATION_HOST"], 
    "port": os.environ["DB_DESTINATION_PORT"],
    "dbname": os.environ["DB_DESTINATION_NAME"],
    "user": os.environ["DB_DESTINATION_USER"],
    "password": os.environ["DB_DESTINATION_PASSWORD"],
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "clean_users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [4]:
df

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,14,5918-VUKWP,2017-06-01,NaT,One year,No,Bank transfer (automatic),20.55,654.55,Fiber optic,...,No,No,No,No,Female,0,No,No,No,0
1,15,1744-JHKYS,2017-04-01,NaT,Month-to-month,No,Electronic check,24.70,780.20,Fiber optic,...,No,No,No,No,Female,0,Yes,No,Yes,0
2,16,2984-RGEYA,2014-05-01,NaT,Two year,No,Bank transfer (automatic),19.75,1375.40,Fiber optic,...,No,No,No,No,Female,0,Yes,Yes,No,0
3,17,9680-NIAUV,2014-02-01,NaT,Two year,No,Credit card (automatic),109.70,8129.30,Fiber optic,...,Yes,No,Yes,Yes,Female,0,Yes,Yes,Yes,0
4,18,2146-EGVDT,2015-03-01,NaT,Two year,No,Credit card (automatic),19.30,1192.70,Fiber optic,...,No,No,No,No,Male,0,Yes,Yes,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7014,7015,6502-MJQAE,2019-11-01,2019-12-01,Month-to-month,Yes,Electronic check,69.60,69.60,Fiber optic,...,No,No,No,No,Male,0,No,No,No,1
7015,7016,6257-DTAYD,2014-03-01,NaT,Two year,Yes,Credit card (automatic),104.15,7365.30,Fiber optic,...,No,Yes,Yes,Yes,Male,0,Yes,No,Yes,0
7016,7017,4616-ULAOA,2014-09-01,NaT,Two year,Yes,Credit card (automatic),110.80,7245.90,Fiber optic,...,Yes,Yes,Yes,Yes,Female,0,Yes,Yes,Yes,0
7017,7018,7693-LCKZL,2019-05-01,2019-10-01,Month-to-month,Yes,Electronic check,80.15,385.00,Fiber optic,...,No,No,No,No,Male,0,Yes,Yes,Yes,1


In [5]:
#import joblib
#with open('model_churn/fitted_model.pkl', 'rb') as f:
#        model=joblib.load(f)

In [6]:
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(','.join(df.columns.values.tolist()))
   

In [7]:
cat_features = df.select_dtypes(include='object')
potential_binary_features = cat_features.nunique() == 2
binary_cat_features = cat_features[potential_binary_features[potential_binary_features].index]
other_cat_features = cat_features[potential_binary_features[~potential_binary_features].index]
num_features = df.select_dtypes(['float'])

preprocessor = ColumnTransformer(
        [
        ('binary', OneHotEncoder(), binary_cat_features.columns.tolist()),
        ('cat', CatBoostEncoder(return_df=False), other_cat_features.columns.tolist()),
        ('num', StandardScaler(), num_features.columns.tolist())
        ],
        remainder='drop',
        verbose_feature_names_out=False
    )


df_trans = preprocessor.fit_transform(df, df['target']) 

In [8]:
#df = df.drop(columns=["is_apartment", "has_elevator"])
X_train, X_test, y_train, y_test = train_test_split(df_trans, df["target"], test_size=0.3, random_state=3)

In [9]:
model = CatBoostClassifier()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

Learning rate set to 0.02033
0:	learn: 0.6791140	total: 49.4ms	remaining: 49.3s
1:	learn: 0.6660682	total: 50.6ms	remaining: 25.3s
2:	learn: 0.6544251	total: 51.8ms	remaining: 17.2s
3:	learn: 0.6433733	total: 53ms	remaining: 13.2s
4:	learn: 0.6330659	total: 54.2ms	remaining: 10.8s
5:	learn: 0.6235688	total: 55.4ms	remaining: 9.18s
6:	learn: 0.6144955	total: 56.6ms	remaining: 8.03s
7:	learn: 0.6043269	total: 58ms	remaining: 7.19s
8:	learn: 0.5963296	total: 59.3ms	remaining: 6.53s
9:	learn: 0.5882368	total: 60.5ms	remaining: 5.99s
10:	learn: 0.5814931	total: 61.7ms	remaining: 5.55s
11:	learn: 0.5749679	total: 63ms	remaining: 5.19s
12:	learn: 0.5677809	total: 64.4ms	remaining: 4.89s
13:	learn: 0.5608318	total: 65.6ms	remaining: 4.62s
14:	learn: 0.5548484	total: 66.8ms	remaining: 4.39s
15:	learn: 0.5480879	total: 68.1ms	remaining: 4.19s
16:	learn: 0.5417553	total: 69.3ms	remaining: 4.01s
17:	learn: 0.5357738	total: 70.7ms	remaining: 3.85s
18:	learn: 0.5300746	total: 71.9ms	remaining: 3.71s

In [10]:

from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, log_loss
# импортируйте необходимые вам модули

# заведите словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода
_, err1, err2, _ = confusion_matrix(y_test, prediction,normalize='all').ravel()
#auc = roc_auc_score(y_test, probas)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
#metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss


In [11]:
"""
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
metrics={}

metrics["mae"] = mean_absolute_error(y_test, prediction)
metrics["mape"] = mean_absolute_percentage_error(y_test, prediction)
"""

'\nfrom sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error\nmetrics={}\n\nmetrics["mae"] = mean_absolute_error(y_test, prediction)\nmetrics["mape"] = mean_absolute_percentage_error(y_test, prediction)\n'

In [12]:

pip_requirements = "requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata =  {'model_type': 'churn_month'}

In [13]:



experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

if not experiment_id:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) # ваш код здесь

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id # ваш код здесь
    
    model_info = mlflow.catboost.log_model( 
			cb_model=model,
            pip_requirements=pip_requirements,
            signature=signature,
            input_example=input_example,
            metadata=metadata,
            #code_path=code_paths,
            await_registration_for=60,
            artifact_path="models",
            registered_model_name=REGISTRY_MODEL_NAME)

    mlflow.log_metrics(metrics) 
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) # ваш код здесь


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert (run.info.status =='FINISHED')# ваш код здесь

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns.txt') # ваш код здесь
#os.remove('users_churn.csv') # ваш код здесьcsv

Successfully registered model 'real_churn_model_Andrey'.
2024/03/29 16:14:46 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: real_churn_model_Andrey, version 1
Created version '1' of model 'real_churn_model_Andrey'.


In [14]:
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri)
model_predictions = loaded_model.predict(X_test)

#assert model_predictions.dtype == int

print(model_predictions[:10])

[0 0 1 0 0 1 0 0 0 1]


In [15]:
client = mlflow.MlflowClient() 

models = client.search_model_versions(
    filter_string=f"name = '{REGISTRY_MODEL_NAME}'"
)
print(f"Model info:\n {models}")

model_name_1 = models[-1].name
model_version_1 = models[-1].version
model_stage_1 = models[-1].current_stage

model_name_2 = models[-2].name
model_version_2 = models[-2].version
model_stage_2 = models[-2].current_stage


print(f"Текущий stage модели 1: {model_stage_1}")
print(f"Текущий stage модели 2: {model_stage_2}")

# поменяйте статус каждой модели
client.transition_model_version_stage(
    model_name_1, model_version_1, 'production')
client.transition_model_version_stage(
    model_name_2, model_version_2, 'staging')

# переимнуйте модель в реестре
client.rename_registered_model(
    name = REGISTRY_MODEL_NAME, new_name = f'{REGISTRY_MODEL_NAME}_b2c')

Model info:
 [<ModelVersion: aliases=[], creation_timestamp=1711718085444, current_stage='None', description='', last_updated_timestamp=1711718085444, name='real_churn_model_Andrey', run_id='b97b26920b84410cb7bda513bd88da4f', run_link='', source='s3://s3-student-mle-20240228-2fd44f5a96/4/b97b26920b84410cb7bda513bd88da4f/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>]


IndexError: list index out of range

In [ ]:
print(f"Текущий stage модели 1: {model_stage_1}")
print(f"Текущий stage модели 2: {model_stage_2}")

Текущий stage модели 1: None
Текущий stage модели 2: None
